In [1]:
from config.constants import MONGODB_CREDENTIALS, SQL_LEAGUES_CONN, SQL_LEAGUES_ENGINE, API_KEY
from pymongo import MongoClient
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from riotwatcher import RiotWatcher

# MongoDB

In [2]:
mongodb = MongoClient(MONGODB_CREDENTIALS).slds

In [3]:
soloq_m_coll = mongodb.soloq_m
soloq_tl_coll = mongodb.soloq_tl

In [51]:
dict1 = {}
# dict1['gameCreation'] = {}

In [56]:
dict1['gameCreation']['$lte'] = 'potatoes'

KeyError: 'gameCreation'

In [54]:
if dict1['gameCreation']:
    print('hello')

KeyError: 'gameCreation'

In [48]:
dict1['gameCreation']['$gteeee'] = 'meirae'

In [49]:
dict1

{'gameCreation': {'$gteeee': 'meirae'}}

In [31]:
gs = soloq_m_coll.find({'gameCreation': {'$gte': 1526335200000, '$lte': 1526507999000}}, {'_id': 0, 'gameId': 1})

In [32]:
gids = [obj['gameId'] for obj in gs]

In [33]:
len(gids)

356

### Regex in MongoDB with game versions

In [193]:
patch = '8.1'

In [194]:
rgex_patch = '^' + '{}'.format(patch).replace('.', r'\.')

In [195]:
gamevers = [game['gameVersion'] for game in soloq_m_coll.find({'platformId': region, 'gameVersion': {'$regex': rgex_patch}}, {'_id': 0, 'gameVersion': 1})]

In [196]:
pd.Series(gamevers).value_counts()

8.1.214.5847    14
8.1.213.4336    13
dtype: int64

In [ ]:
g = soloq_m_coll.find({'participantIdentities.player.accountId': {'$in': [23787352]}}, {'_id': 0, 'gameId': 1})

In [ ]:
gs = [obj['gameId'] for obj in g]

In [ ]:
pd.DataFrame(gs).rename(columns={0: 'game_id'})

# SQL

In [2]:
engine = create_engine(SQL_LEAGUES_ENGINE)

In [3]:
cnx = engine.connect()

In [4]:
df = pd.read_csv('C:\\Users\\david\\Downloads\\players_accs.csv')

In [5]:
df.to_sql(con=cnx, name='soloq', if_exists='replace')

In [6]:
df1 = pd.read_sql_table(con=cnx, table_name='soloq')

In [7]:
df1.loc[df1.team_abbv == 'KIYF']

,index,region,competition,competition_abbv,team_name,team_abbv,main_role,substitute,player_name,account_id,id,account_type
95,95,EUW1,Superliga Orange,SLO,KIYF,KIYF,ADC,0,Carzzy,225783506,82347795,1
96,96,EUW1,Superliga Orange,SLO,KIYF,KIYF,JUNG,0,Aesthetic,24246914,20902645,1
97,97,EUW1,Superliga Orange,SLO,KIYF,KIYF,MID,0,ZaZee,26290492,22587236,1
98,98,EUW1,Superliga Orange,SLO,KIYF,KIYF,MID,0,ZaZee,237164085,105057222,2
99,99,EUW1,Superliga Orange,SLO,KIYF,KIYF,MID,0,ZaZee,235326169,102870157,2
100,100,EUW1,Superliga Orange,SLO,KIYF,KIYF,SUPP,0,Rufus,38889386,35747269,1
101,101,EUW1,Superliga Orange,SLO,KIYF,KIYF,TOP,0,Orome,203454768,44747167,1


In [8]:
cnx.close()

In [21]:
any(df1.account_id) is None

False